In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import json

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [ ]:
with open("commandline_qa.json", "r", encoding="utf-8") as f:
    data = json.load(f)

questions = [item["question"] for item in data]
answers = [item["answer"] for item in data]

from datasets import Dataset

dataset = Dataset.from_dict({"question": questions, "answer": answers})
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
test_dataset = dataset['test']


In [5]:

MODEL_NAME = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)


`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:

max_length = 256

def preprocess(batch):
    prompts = [f"Question: {q}\nAnswer:" for q in batch["question"]]
    inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=max_length)
    outputs = tokenizer(batch["answer"], truncation=True, padding="max_length", max_length=max_length)
    inputs["labels"] = outputs["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 168/168 [00:00<00:00, 347.77 examples/s]

Map: 100%|██████████| 19/19 [00:00<00:00, 238.26 examples/s]


In [7]:

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


trainable params: 294,912 || all params: 125,493,504 || trainable%: 0.2350


In [ ]:
training_args = TrainingArguments(
    output_dir="./lora-gptneo",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,  
    save_total_limit=2,
    remove_unused_columns=False,
)

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_27628\4074003995.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [9]:
prompt = "How do I check if a variable is set in Bash?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "", 1).strip()
print("Generated Answer:\n", answer)


Generated Answer:
 I have a script that checks if a variable is set in Bash.

A:

You can use the -f option to check if the variable is set.
$ echo "variable set" | grep -q "variable set"
variable set

A:

You can use the -f option to check if the variable is set.
$ echo "variable set" | grep -q "variable set"
variable set

A:

You can use the -f option to check if the variable is set.
$ echo "variable set" | grep -q "variable


In [ ]:
import os
from peft import PeftModel

ckpt_dir = "./lora-gptneo/checkpoint-90"
possible_paths = [ckpt_dir, "lora-gptneo/checkpoint-90", os.path.join(os.getcwd(), "lora-gptneo", "checkpoint-90")]
found_path = None
for p in possible_paths:
    if os.path.exists(p):
        found_path = p
        break

if found_path:
    try:
        loaded_model = PeftModel.from_pretrained(base_model, found_path)
        loaded_model = loaded_model.to(device)
        print(f"Loaded LoRA adapter from {found_path} and moved to {device}")
    except Exception as e:
        print("Failed to load LoRA adapter from checkpoint:", e)
        loaded_model = model  
else:
    print(f"No checkpoint found at {possible_paths}; using the current model")
    loaded_model = model

num_examples = 5
if len(dataset['test']) < num_examples:
    num_examples = len(dataset['test'])

for i in range(num_examples):
    q = dataset['test'][i]["question"]
    ref = dataset['test'][i]["answer"]
    prompt = f"Question: {q}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length).to(loaded_model.device)
    with torch.no_grad():
        outputs = loaded_model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            early_stopping=True
        )
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "", 1).strip()
    print("\n--- Example", i+1, "---")
    print("Question:", q)
    print("Reference:\n", ref)
    print("Generated:\n", ans)


No checkpoint found at ['./lora-gptneo/checkpoint-90', 'lora-gptneo/checkpoint-90', 'c:\\Users\\KIIT0001\\Desktop\\StackoverflowQ&A\\lora-gptneo\\checkpoint-90']; using the current model

--- Example 1 ---
Question: How to concatenate string variables in Bash
Reference:
 foo="Hello"
foo="${foo} World"
echo "${foo}"
> Hello World


In general to concatenate two variables you can just write them one after another:

a='Hello'
b='World'
c="${a} ${b}"
echo "${c}"
> Hello World
Generated:
 A:

You can do it like this:
#!/usr/bin/env bash

echo $1
echo $2
echo $3
echo $4
echo $5
echo $6
echo $7
echo $8
echo $9
echo $10
echo $11
echo $12
echo $13
echo $14

--- Example 2 ---
Question: How do I iterate over a range of numbers defined by variables in Bash?
Reference:
 for i in $(seq 1 $END); do echo $i; done

edit: I prefer seq over the other methods because I can actually remember it ;)
Generated:
 If you want to iterate over a range of numbers defined by variables in Bash, you can do it like th

In [ ]:
num_examples = 3
for i in range(num_examples):
    q = dataset['test'][i]["question"]
    ref = dataset['test'][i]["answer"]
    prompt = f"Question: {q}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length).to(loaded_model.device)
    with torch.no_grad():
        outputs = loaded_model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            early_stopping=True
        )
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "", 1).strip()
    ans_trim = (ans[:400] + '...') if len(ans) > 400 else ans
    print(f"\n-- Example {i+1} --\nQuestion: {q}\nReference: {ref[:400]}\nGenerated: {ans_trim}\n")



-- Example 1 --
Question: How to concatenate string variables in Bash
Reference: foo="Hello"
foo="${foo} World"
echo "${foo}"
> Hello World


In general to concatenate two variables you can just write them one after another:

a='Hello'
b='World'
c="${a} ${b}"
echo "${c}"
> Hello World
Generated: $ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_completion/bash_completion.sh
$ cat /etc/bash_com


-- Example 2 --
Question: How do I iterate over a range of numbers defined by variables in Bash?
Reference: for i in $(seq 1 $END); do echo $i; done

edit: I prefer seq over the other methods because I can actually remember it ;)
Generated: If you want to iterate over a range of numbers, you can do it like this:
for i in range(1,10):
    for j in range(1,1

In [ ]:
trainer.args.max_steps = 50
trainer.args.num_train_epochs = 1
trainer.args.logging_steps = 10
trainer.args.save_strategy = 'steps'
trainer.args.save_steps = 25

trainer.train_dataset = train_dataset

train_info = trainer.train()
print("Training finished:\n", train_info)

trainer.save_model(trainer.args.output_dir)
try:
    model.save_pretrained(trainer.args.output_dir)
    print('model.save_pretrained() success')
except Exception as e:
    print('model.save_pretrained() failed:', e)

try:
    model.save_pretrained(trainer.args.output_dir)
    print('PeftModel.save_pretrained() success')
except Exception as e:
    print('PeftModel.save_pretrained() failed:', e)

import os
print('lora-gptneo contents after training:', os.listdir('lora-gptneo'))


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
c:\Users\KIIT0001\Desktop\StackoverflowQ&A\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\KIIT0001\Desktop\StackoverflowQ&A\.venv\Lib\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
c:\Users\KIIT0001\Desktop

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
import os
print('trainer.args.output_dir:', trainer.args.output_dir)
print('trainer.args.save_strategy:', trainer.args.save_strategy)
print('trainer.args.save_steps:', getattr(trainer.args, 'save_steps', None))
print('trainer.state.global_step:', trainer.state.global_step)
print('trainer.state.max_steps:', trainer.state.max_steps)
print('trainer.state.epoch:', trainer.state.epoch)
print('is_world_process_zero:', trainer.is_world_process_zero())

print('\nCWD:', os.getcwd())
print('lora-gptneo exists:', os.path.exists('lora-gptneo'))
print('lora-gptneo contents before save:', os.listdir('lora-gptneo'))

# Force save the adapter + trainer state
print('\nSaving model and adapter now to', trainer.args.output_dir)
trainer.save_model(trainer.args.output_dir)
try:
    model.save_pretrained(trainer.args.output_dir)
    print('model.save_pretrained() success')
except Exception as e:
    print('model.save_pretrained() failed:', e)

try:
    model.save_pretrained(trainer.args.output_dir)
    print('PeftModel.save_pretrained() success')
except Exception as e:
    print('PeftModel.save_pretrained() failed:', e)

print('lora-gptneo contents after save:', os.listdir('lora-gptneo'))


trainer.args.output_dir: ./lora-gptneo
trainer.args.save_strategy: SaveStrategy.EPOCH
trainer.args.save_steps: 500
trainer.state.global_step: 10
trainer.state.max_steps: 10
trainer.state.epoch: 1.0
is_world_process_zero: True

CWD: c:\Users\KIIT0001\Desktop\StackoverflowQ&A
lora-gptneo exists: True
lora-gptneo contents before save: ['checkpoint-10']

Saving model and adapter now to ./lora-gptneo
model.save_pretrained() success
model.save_pretrained() success
PeftModel.save_pretrained() success
lora-gptneo contents after save: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-10', 'merges.txt', 'README.md', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.json']
PeftModel.save_pretrained() success
lora-gptneo contents after save: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-10', 'merges.txt', 'README.md', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.js

trainer.args.output_dir: ./lora-gptneo
trainer.args.save_strategy: steps
trainer.args.save_steps: 100
trainer.state.global_step: 9
trainer.state.max_steps: 200
trainer.state.epoch: 0.10714285714285714
is_world_process_zero: True

CWD: c:\Users\KIIT0001\Desktop\StackoverflowQ&A
lora-gptneo exists: True
lora-gptneo contents before save: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-10', 'config.json', 'merges.txt', 'README.md', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.json']

Saving model and adapter now to ./lora-gptneo
model.save_pretrained() success
model.save_pretrained() success
PeftModel.save_pretrained() success
lora-gptneo contents after save: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-10', 'config.json', 'merges.txt', 'README.md', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.json']
PeftModel.save_pretrained() success
lora-gptneo c

In [ ]:
import os
os.makedirs('lora-gptneo', exist_ok=True)

try:
    model.save_pretrained('lora-gptneo')
    print('PeftModel.save_pretrained: success')
except Exception as e:
    print('PeftModel.save_pretrained: failed ->', e)

try:
    base_model.config.save_pretrained('lora-gptneo')
    print('base_model.config.save_pretrained: success')
except Exception as e:
    print('base_model.config.save_pretrained: failed ->', e)

print('lora-gptneo contents:', os.listdir('lora-gptneo'))


PeftModel.save_pretrained: success
base_model.config.save_pretrained: success
lora-gptneo contents: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-10', 'config.json', 'merges.txt', 'README.md', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'training_args.bin', 'vocab.json']


In [ ]:
import os
for root, dirs, files in os.walk('.', topdown=True):

    if root == '.':
        print('ROOT children:', dirs)
    if root.startswith('./lora-gptneo') and root.count(os.sep) <= 2:
        print('LORA tree:', root, dirs, files)


ROOT children: ['.venv', 'logs', 'lora-gptneo']
